## 从零开始实现

In [2]:
%matplotlib inline
import math
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

## 数据读取

读入数据，`num_steps` 就是一个小批量的样本长度，即 T

In [3]:
batch_size, num_steps = 32, 35
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps, token_name='char')

### one-hot 编码

one-hot 编码将下标转变为深度学习处理的张量，因为输入是词元，要通过下标转换转变为张量

In [4]:
F.one_hot(torch.tensor([0, 2]), len(vocab)) # 将下标变为向量，输出是 len(list:index) * len(vocab) 

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]])

小批量的形状是（批量大小，时间步数），one-hot 处理的时候对小批量进行转置，形状变成（num_steps, batch_size），one-hot 最终输出会变成（num_steps, batch_size, len(vocab)），便于我们同时处理多个样本的相同序列，即访问所有某一时刻 x(t) 的数据

结合 X 的形状来说，原先每个批量是有 32 个样本，每个样本有 35 个时间步长，转置后是，有 35 个时间步长，每个步长有两个样本（样本间互不干扰），转置前神经网络按样本逐步处理，转置后按照时间顺序处理，比如先处理第一个时间步长的 32 个数据，再处理下一个时间步长

有一个误区，由于不同样本的时间步长在原序列上相邻，可能会认为样本间有联系，其实样本间没有联系，都是作为独立样本训练，而样本包含标签和特征，标签是根据特征生成，而标签的长度要和使用的 **N元语法** 算法相关

In [5]:
for X, y in train_iter:
    print(F.one_hot(X.T, len(vocab)).shape)
    break

torch.Size([35, 32, 28])


## 初始化模型参数

初始化生成隐藏层和输出层的权重

In [6]:
def init_params(vocab_size: int, num_hiddens: int, device: any=None) -> list:
    num_inputs = num_outputs = vocab_size # 输入和输出都是词元，词元又表示成下标张量，即都为 vocab_size

    def normal(shape):
        return torch.randn(size=shape, device=device) * 0.01
    
    # 隐藏层
    W_xh = normal((num_inputs, num_hiddens)) # 从输入到神经元，正常的网络都有
    W_hh = normal((num_hiddens, num_hiddens)) # 两个时间的神经元相关联，RNN 特有，把这一行删除就是一个单隐藏层的 MLP
    b_h = torch.zeros(num_hiddens, device=device)

    # 输出层
    W_hq = normal((num_hiddens, num_outputs))
    b_q = torch.zeros(num_outputs, device=device)

    # 权重附加梯度
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.requires_grad_(True)
    return params

### 循环神经网络

0 时刻的时候没有上一个时刻的隐藏状态，`init_rnn_state` 返回 0 时刻的隐藏状态，形状是（批量大小，神经元数量），返回的类型是 `tuple`，目前只有一个东西，后面的 `LSTM` 可能有多个

In [7]:
def init_rnn_state(batch_size: int, num_hiddens: int, device: any=None) -> tuple:
    return (torch.zeros((batch_size, num_hiddens), device=device))

`rnn` 函数在一个时间步内计算隐藏状态和输出，最终输出形状是（（num_steps * batch_size），len(vocab)），即整合所有时间步的输出，方便后续操作，以及输出最终的隐藏状态，可以作为下一个序列的初始状态，在语言模型中就是记忆你之前说的话

In [8]:
def rnn(inputs: torch.tensor, state: tuple, params: list) -> torch.tensor:
    """计算一个批量的隐藏状态和输出
    """
    W_xh, W_hh, b_h, W_hq, b_q = params
    H = state
    outputs = []

    for X in inputs: # 循环最外层（时间步）
        H = torch.tanh(torch.mm(X, W_xh) + torch.mm(H, W_hh) + b_h) # 当前时刻的隐藏状态
        Y = torch.mm(H, W_hq) + b_q # 当前时刻的输出，用于预测下一时刻的输入（计算损失）
        outputs.append(Y) # outputs 列表内有 num_steps 个元素，代表每个时间步数的不同样本的输出
    return torch.cat(outputs, dim=0), (H, )

以上的 `rnn, init_rnn_state, init_params` 函数都会整合到 `RNNModelScratch` 类中，以上只是方便解释

In [29]:
class RNNModelScratch:
    """从零开始实现的循环神经网络模型"""
    def __init__(self, vocab_size: int, num_hiddens: int, device: any, get_params: any, init_state: any, forward_fn: any):
        self.vocab_size, self.num_hiddens = vocab_size, num_hiddens
        self.params = get_params(vocab_size, num_hiddens, device)
        self.init_state, self.forward_fn = init_state, forward_fn

    def __call__(self, X, state):
        X = F.one_hot(X.T, self.vocab_size).type(torch.float32)
        return self.forward_fn(X, state, self.params)

    def begin_state(self, batch_size, device):
        return self.init_state(batch_size, self.num_hiddens, device)

In [32]:
num_hiddens = 512
X = torch.arange(10).reshape(2, 5)
net = RNNModelScratch(len(vocab), num_hiddens, d2l.try_gpu(), init_params, init_rnn_state, rnn)
state = net.begin_state(X.shape[0], d2l.try_gpu())
Y, new_state = net(X.to(d2l.try_gpu()), state)
Y.shape, len(new_state), new_state[0].shape

(torch.Size([10, 28]), 1, torch.Size([2, 512]))